# Mnist手写数字识别

In [1]:
import tensorflow as tf
import numpy as np

## 数据

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 参数

In [3]:
chunk_size = 28
chunk_n = 28

rnn_size = 256

n_output_layer = 10

### placeholder（占位符）(dtype=tf.float32, shape=[144, 10], name='X')
dtype：数据类型，必填，默认为value的数据类型，传入参数为tensorflow下的枚举值（float32，float64.......）

shape：数据形状，选填，不填则随传入数据的形状自行变动，可以在多次调用中传入不同形状的数据

name：常量名，选填，默认值不重复，根据创建顺序为（Placeholder，Placeholder_1，Placeholder_2.......）

占位符是tf流程图的起始节点，另外，RNN状态向量也是存储在占位符中，在每一次运行后更新输出

In [4]:
X = tf.placeholder('float', [None, chunk_n, chunk_size])
Y = tf.placeholder('float')

## Variable 通过Variable（注：V大写）方法创建，并且需要传递初始值。在使用前需要通过tensorflow的初始化方法进行初始化。
W = tf.Variable(initial_value=tf.zeros([9, 5]),  # 初始值，必填，张量或可以转换为张量的Python对象。初始值必须有指定一个形状，除非`validate_shape`设置为False。（形状9,5）
                
                trainable=True,  # 如果`True`，则默认值也将变量添加到图形中集合`GraphKeys.TRAINABLE_VARIABLES`。这个集合用作“Optimizer”类使用的默认变量列表
                
                collections=None,  # 图表集合键的列表。新的变量被添加到这些集合。默认为`[GraphKeys.GLOBAL_VARIABLES]`。
                
                validate_shape=True,  # 如果`False`，允许变量用初始化未知形状的值。如果“True”，默认的形状`initial_value`必须是已知的。
                
                caching_device=None,  # 可选设备字符串，描述变量的位置应该被缓存以供阅读。默认为变量的设备。如果不是“None”，则缓存在另一个设备上。典型的用途是缓存在使用变量的Ops所在的设备上进行重复数据删除复制`Switch`和其他条件语句。
                
                name='W',  # 变量的可选名称。默认为“Variable”并获取自动去重（Variable_1,Variable_2....）。
                
                variable_def=None, # `VariableDef`协议缓冲区。如果不是“无”，则重新创建变量对象及其内容，引用变量的节点在图中，必须已经存在。图形没有改变。`variable_def`和其他参数是互斥的。
                
                dtype=tf.float32, # 如果设置，initial_value将被转换为给定的类型。如果`None'，数据类型将被保存（如果`initial_value`是一个张量），或者“convert_to_tensor”来决定。
                
                expected_shape=None,  # 张量的Shape。如果设置，initial_value需要符合这个形状。
                
                import_scope=None)  # 可选的字符串。名称范围添加到`Variable.`仅在从协议缓冲区初始化时使用。

##### transpose(a, perm=None, name="transpose"):

perm:控制转置的操作,以perm = [0,1,2] 3个维度的数组为例, 0--代表的是最外层的一维, 1--代表外向内数第二维, 2--代表最内层的一维,这种perm是默认的值

#### tf.reshape(data, [-1, chunk_size])
-1代表自动计算

#### tf.split(dimension, num_split, input)：
dimension的意思就是输入张量的哪一个维度，如果是0就表示对第0维度进行切割。num_split就是切割的数量，如果是2就表示输入张量被切成2份，每一份是一个列表。

#### 单层rnn：tf.contrib.rnn.static_rnn：

输入：[步长,batch,input] 
输出：[n_steps,batch,n_hidden] 

#### 多层rnn：tf.nn.dynamic_rnn：

输入：[batch,步长,input] 
输出：[batch,n_steps,n_hidden] 

In [5]:
def recurrent_neural_network(data):##data:训练集 无标签样本
      layer = {'w_':tf.Variable(tf.random_normal([rnn_size, n_output_layer])), 'b_':tf.Variable(tf.random_normal([n_output_layer]))}
  
      lstm_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
      print(data.shape)
      
      data = tf.transpose(data, [1,0,2])#为数组的转置函数，转置最外两层
      
      print(data.shape)
      
      data = tf.reshape(data, [-1, chunk_size])##-1代表自动计算
    
      data = tf.split(data, chunk_n)##向量分割
    
      outputs, status = tf.contrib.rnn.static_rnn(lstm_cell, data, dtype=tf.float32)
  
      ouput = tf.add(tf.matmul(outputs[-1], layer['w_']), layer['b_'])
    #output=outputs[-1] 就取到了最后一步的ouput
    #matmul 矩阵相乘
    #add矩阵相加
      return ouput 

In [6]:
batch_size = 100

#### tf.reduce_mean(input_tensor, axis=None, keep_dims=False, name=None, reduction_indices=None)
可跨越维度的计算张量各元素的平均值
#### softmax_cross_entropy_with_logits(_sentinel=None, # pylint: disable=invalid-name,labels=None,logits=None,dim=-1, name=None)      
_sentinel：该参数为内部使用，实际运用中不会使用。目的是占据函数参数的第一个位置，让我们不能通过位置传递labels 和 logits，而是使用关键字传递参数，避免混淆
labels: 实际标签
logits: 对数几率，需要注意的是这个函数内部自动计算 softmax，然后再计算交叉熵代价函数，也就是说 logits 必须是没有经过 tf.nn.softmax 函数处理的数据，否则导致训练结果有问题。
#### AdamOptimizer
___init__(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
                                      

In [7]:
def train_neural_network(X, Y):
    predict = recurrent_neural_network(X)
    cost_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=predict))##preddict 一定没有经过softmax
    optimizer = tf.train.AdamOptimizer().minimize(cost_func)
    epochs = 10
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(epochs):
                epoch_loss = 0
                for i in range(int(mnist.train.num_examples/batch_size)):
                    x,y = mnist.train.next_batch(batch_size)
                    x = x.reshape([batch_size,chunk_n,chunk_size])
                    _,c = sess.run([optimizer,cost_func],feed_dict = {X:x,Y:y})
                    epoch_loss += c
                print(epoch,':',epoch_loss)
#                 acc = tf.reduce_mean(tf.cast(correct,'float'))
#                 print('accuracy: ',acc.eval({X:mnist.test.images.reshape(-1,chuk_n,chunk_size),Y:mnist.test.lables}))
        print('pre',predict.shape,'\n',predict[-1],'\n','Y',Y.shape)
        
        correct = tf.equal(tf.argmax(predict,1), tf.argmax(Y,1))
        acc = tf.reduce_mean(tf.cast(correct,'float'))
        print('accuracy: ',acc.eval({X:mnist.test.images.reshape(-1,chunk_n,chunk_size),Y:mnist.test.labels}))

In [8]:
train_neural_network(X,Y)

(?, 28, 28)
(28, ?, 28)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
0 : 210.895644974
1 : 66.2925029853
2 : 43.7067454709
3 : 34.1231181358
4 : 27.134102992
5 : 22.7766535276
6 : 19.1672370344
7 : 17.2465875364
8 : 14.9541022436
9 : 12.6869191339
pre (?, 10) 
 Tensor("strided_slice:0", shape=(10,), dtype=float32) 
 Y <unknown>
accuracy:  0.9852


### Scope
#### Name_scope:
with tf.name_scope(name):
在这里创建的Variable通过tf.Varialble 创建的Variable都会在Variable 前加入 name/variable_name
表示在name底下创建了variable
并且如果创建时只定variavble_name在同namescope下会使viriable自动修改名字

#### Variable_scope:
通过tf.get_variable创建也会有名字
并且想要重复使用某一个Variable可以通过tf.get_variable（name）只输入名字就可以调用前面的变量了
并且在这个重复利用的语句前面需要嗲用scope（with .... as scope）.resuse_variable()

Variable 创建方式：tf.get_varialble:需要指定initializer tf.Varialble

为什么需要使用Scope：RNN需要循环利用
例：training 和 Testing中的网络结构不同但是，test中需要使用train的Variable

首先在同一个scope下创建好trainRNN，然后使用testRNN创建，创建过程中所有的Variable参数在